In [4]:
# Document loading, retrieval methods and text splitting
%pip install -qU langchain langchain_community

# Local vector store via Chroma
%pip install -qU langchain_chroma

# Local inference and embeddings via Ollama
%pip install -qU langchain_ollama

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install -qU langchain-mistralai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [23]:
!pip install --quiet langchain_experimental langchain_openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.4.2 which is incompatible.
streamlit 1.26.0 requires packaging<24,>=16.8, but you have packaging 24.1 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 1.2 MB/s eta 0:00:00 MB/s eta 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [21]:
from langchain_community.document_loaders import PyPDFLoader

'''def load_data(path):
    loader = PyPDFLoader(file_path)
    docs_data = loader.load()
    return docs_data'''

file_path = "invest4g_Product_Brochure.pdf"
fp_iselect="product-brochure.pdf"
fp_health="Health_First_Plan_Brochure_Final.pdf"
fp_ytp="ytp-product-brochure-v02.pdf"
loader1 = PyPDFLoader(file_path)
loader2 = PyPDFLoader(fp_iselect)
loader3 = PyPDFLoader(fp_health)
loader4 = PyPDFLoader(fp_ytp)
docs=loader1.load()
iselect_docs=loader2.load()
health_docs=loader3.load()
ytp_docs=loader4.load()
print(len(docs))
print(len(iselect_docs))
print(len(health_docs))
print(len(ytp_docs))

28
31
19
41


In [10]:
import pypdf
from tqdm import tqdm
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = pypdf.PdfReader(file)
        text = ""
        for page in tqdm(reader.pages, desc="Extracting PDF"):
            text += page.extract_text()
    return text

invest_4g_docs=extract_text_from_pdf("invest4g_Product_Brochure.pdf")

Extracting PDF: 100%|███| 28/28 [00:00<00:00, 28.76it/s]


In [3]:
print(docs[1].metadata['source'])
print(iselect_docs[1].metadata['source'])
print(health_docs[1].metadata['source'])
print(ytp_docs[1].metadata['source'])

invest4g_Product_Brochure.pdf
product-brochure.pdf
Health_First_Plan_Brochure_Final.pdf
ytp-product-brochure-v02.pdf


In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

# Use a local HuggingFace model instead
local_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
text_splitter = SemanticChunker(local_embeddings,breakpoint_threshold_type="gradient")

In [13]:
docs = text_splitter.create_documents([invest_4g_docs])
print(docs[0].page_content)

1There are no charges other than Fund Management Charge, Mortality Charge and  Premium Funding Benefit Char ge (in case of Care Option) that would be 
deducted.2Refund 
of Mortality Charge is available in all three options.3Additions starting from the end of every 5th policy year from the commencement of the policy 
and is available till the end of Premium Payment Term, provided all the due premiums are received till that time. 4Additional additions at the end of every 5th policy year 
starting from 10th policy year, provided all the due premiums till date have been paid. The Unit Linked Insurance Products do not  offer any liquidity during the
first five years of the contract. The Policyholder will not be able to surrender/withdraw the monies invested in Unit Linked 
Insurance Products completely or partially till the end of the fifth year. Purchase of any insurance products by a bank’s customer is purely voluntary and is not 
linked to availment of any other facility from the bank. S

In [14]:
print(len(docs))

31


In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def return_data_splits(docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=256)
    all_splits = text_splitter.split_documents(docs)
    return all_splits

fourg_chunks=return_data_splits(docs)
iselect_chunks=return_data_splits(iselect_docs)
health_chunks=return_data_splits(health_docs)
ytp_chunks=return_data_splits(ytp_docs)
print(len(fourg_chunks))
print(len(iselect_chunks))
print(len(health_chunks))
print(len(ytp_chunks))

56
256
48
574


In [7]:
all_chunks=fourg_chunks+iselect_chunks+health_chunks+ytp_chunks
len(all_chunks)

934

In [ ]:
pip install sentence-transformers

In [15]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Use a local HuggingFace model instead
local_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vectorstore = Chroma.from_documents(documents=docs, embedding=local_embeddings)


In [21]:
retriever=vectorstore.as_retriever()

In [16]:
question = "What are the potential impacts on the death benefit and fund value under the Century Option if the policyholder has made extensive use of the SWO feature, particularly in policies with a long-term horizon and high equity exposure?"
docs = vectorstore.similarity_search(question,k=5)
len(docs)

5

In [12]:
rel_docs=vectorstore.similarity_search_with_score(question,k=3)
print(rel_docs)

[(Document(metadata={'page': 0, 'source': 'invest4g_Product_Brochure.pdf'}, page_content='1There are no charges other than Fund Management Charge, Mortality Charge and  Premium Funding Benefit Char ge (in case of Care Option) that would be \ndeducted.2Refund \nof Mortality Charge is available in all three options.3Additions starting from the end of every 5th policy year from the commencement of the policy \nand is available till the end of Premium Payment Term, provided all the due premiums are received till that time. 4Additional additions at the end of every 5th policy year \nstarting from 10th policy year, provided all the due premiums till date have been paid. The Unit Linked Insurance Products do not  offer any liquidity during the\nfirst five years of the contract. The Policyholder will not be able to surrender/withdraw the monies invested in Unit Linked \nInsurance Products completely or partially till the end of the fifth year. Purchase of any insurance products by a bank’s cus

In [13]:
docs[0].page_content

'1There are no charges other than Fund Management Charge, Mortality Charge and  Premium Funding Benefit Char ge (in case of Care Option) that would be \ndeducted.2Refund \nof Mortality Charge is available in all three options.3Additions starting from the end of every 5th policy year from the commencement of the policy \nand is available till the end of Premium Payment Term, provided all the due premiums are received till that time. 4Additional additions at the end of every 5th policy year \nstarting from 10th policy year, provided all the due premiums till date have been paid. The Unit Linked Insurance Products do not  offer any liquidity during the\nfirst five years of the contract. The Policyholder will not be able to surrender/withdraw the monies invested in Unit Linked \nInsurance Products completely or partially till the end of the fifth year. Purchase of any insurance products by a bank’s customer is purely voluntary and is not \nlinked to availment of any other facility from the

In [17]:
docs

[Document(page_content='However, any re-activation or stopping of this feature will take place only from the Policy Anniversary following the receipt of the request for the same. viii. Only one of ‘Milestone Withdrawal Option’ or ‘Systematic Withdrawal Option’ can be exercised in a Policy. Either of the two options can be chosen and changed up to anytime before the payout starts. Once the payout from either of the options has started, the Policyholder cannot switch his withdrawal option or opt for other withdrawal option at the same time. Policyholder can opt out of this option at anytime even after the withdrawals have started but will not be allowed to opt for other withdrawal option available in the Plan. ix. SWO percentage can be changed anytime during the Policy Term, even after SWO has become operational. The change in percentage will be eﬀective from the next Policy Anniversary following the receipt of the request for the same. x. The SWO payout will be payable till the end of t

In [4]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:01:01
Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

#local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

db = FAISS.from_documents(docs, local_embeddings)

In [19]:
question = "What are the potential impacts on the death benefit and fund value under the Century Option if the policyholder has made extensive use of the SWO feature, particularly in policies with a long-term horizon and high equity exposure?"
docs = db.similarity_search(question,k=5)
len(docs)

5

In [13]:
docs[0].page_content

'Until the expiry of Grace Period, the Policy is deemed to be in-force with beneﬁts and applicable charges continuing as per \nterms and conditions of the Policy. \nFor an in force Policy, if a surrender request is received within the ﬁrst 5 policy years, the Fund Value after deduction of \napplicable Surrender Charges is transferred to the DPF and the proceeds of discontinued policy shall be refunded to the Policyholder only after completion of Lock-in Period.\nF. Discontinuance of Policy after the Lock-in Period (after ﬁrst ﬁve years):\nIf the Premium is not received by the expiry of the Grace Period, the Policy shall be converted into a Reduced Paid-up \nPolicy, with the Paid-up Sum Assured. The Policy shall continue to be in Reduced Paid-up status. All applicable charges'

In [20]:
docs

[Document(page_content='However, any re-activation or stopping of this feature will take place only from the Policy Anniversary following the receipt of the request for the same. viii. Only one of ‘Milestone Withdrawal Option’ or ‘Systematic Withdrawal Option’ can be exercised in a Policy. Either of the two options can be chosen and changed up to anytime before the payout starts. Once the payout from either of the options has started, the Policyholder cannot switch his withdrawal option or opt for other withdrawal option at the same time. Policyholder can opt out of this option at anytime even after the withdrawals have started but will not be allowed to opt for other withdrawal option available in the Plan. ix. SWO percentage can be changed anytime during the Policy Term, even after SWO has become operational. The change in percentage will be eﬀective from the next Policy Anniversary following the receipt of the request for the same. x. The SWO payout will be payable till the end of t

In [23]:
rel_docs=[]
for i in docs:
    rel_docs.append(i.metadata['source'])
print(set(rel_docs))
print()
#print(rel_docs)
print()
print("select the product below from the given options with which you want to get the answer to your question")

print()
req_docs=[]
for i in docs:
    if i.metadata['source']=='product-brochure.pdf':
        req_docs.append(i)

print(req_docs)

{'invest4g_Product_Brochure.pdf', 'product-brochure.pdf'}


select the product below from the given options with which you want to get the answer to your question

[Document(metadata={'source': 'product-brochure.pdf', 'page': 17}, page_content="Upon payment of Surrender Value in respect of a life, all beneﬁts attaching to that life \nunder this Policy will cease.\nThe Policy acquires a Guaranteed Surrender Value (GSV) after payment of at least ﬁrst \n2 consecutive Policy Years' Premiums.\nThe GSV shall be equal to, subject to minimum being zero:B * Total Premiums Paid less cumulative Survival Monthly Income already paid, (if any)Where Factor B is guaranteed for the entire Policy Term.  For the details on Factor B \napplicable, please refer to the sample policy contract of this plan available on the Company’s website.\nThe Policy acquires a Special Surrender Value (SSV) after payment of at least ﬁrst 2 \nconsecutive Policy Years' Premiums. \nThe SSV shall be equal to, subject to minimum

In [22]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.1:8b",
)

In [17]:
from langchain_mistralai import ChatMistralAI

# Prompt for the API key securely

# Initialize the ChatMistralAI with the API key directly/
llm = ChatMistralAI(
    model="mistral-large-latest",
    mistral_api_key="cfX9dJSmw8eZAo2K7TvEVR3aiyc2Crvb"
)


In [20]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
    "You are a very helpful assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, reply in a polite way"
    "by saying please ask the relevant questions. Use four sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [23]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
    "You are a very helpful assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, reply in a polite way"
    "by saying please ask the relevant questions. Use four sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain1 = create_stuff_documents_chain(model, prompt)
rag_chain1 = create_retrieval_chain(retriever, question_answer_chain1)


In [19]:
questions=['What charges, other than the Fund Management Charge and Mortality Charge, might be deducted in the given context and under what conditions?',
 'What is the policy regarding surrender/withdrawal of monies invested in Unit Linked Insurance Products during the first five years of the contract?',
 'What are the three options available under the Invest 4G plan offered by Canara HSBC Life Insurance?',
 'Who bears the investment risk in the investment portfolio of the Invest 4G policy, according to the provided context?',
 'What specific life insurance plan is being introduced in the context, and what unique benefits does it offer to policyholders?',
 'According to the context, what different life stages does the mentioned life insurance plan cater to, and how does it address the financial planning needs at each stage?',
 'What are the different cover options available in the Canara HSBC Life Insurance Invest 4G plan, and how do they cater to different life stages?',
 "How does the Premium Funding Benefit feature in the Canara HSBC Life Insurance Invest 4G plan ensure that the targeted savings contributions are made even in the policyholder's absence?",
 'How does the policy ensure that the targeted savings contributions are made even in the absence of the policyholder?',
 'At what specific policy years are the Wealth Boosters added to the Fund Value and how long does this benefit continue till?',
 'What are the different premium paying options available in this policy and how flexible are they?',
 'Can you explain the concept of Wealth Boosters in this policy and when are they awarded during the policy term?',
 'What specific charges are included under the Risk Charges as per the Income Tax Act, 1961, as amended from time to time?',
 'Under which specific option are Premium Funding Benefit (PFB) charges applicable, according to the provided context?',
 'What is the age range for entering the Life Option Cover and what is the policy term for this cover option?',
 'How does the Century Option differ from the Life Option in terms of maturity age and policy term for Regular Pay?',
 'What happens to the policy under the Care Option if the Life Assured passes away before the maturity of the policy?',
 'How is the Fund Value determined and when is it payable under the Sum Assured Plus option?',
 'How does the Century Option handle the payout upon the death of the Life Assured, considering partial withdrawals and the Fund Value?',
 'In the context of the given option that pays a lump sum on the death of the Life Assured, how are the remaining premiums handled, and what happens to the Fund Value upon policy maturity?',
 'What happens to the policy and the fund value when the life assured reaches 100 years of age, according to the given context?',
 'Can the Cover Option be changed after the inception stage during the Policy Term as per the provided information?',
 'What are the different Premium Payment Modes available in this plan and what is the definition of age used for the given entry ages?',
 'Could you explain the Sum Assured for Single Pay and Regular Pay options in terms of their limits and basis age?',
 'How does the policy define the age of the policyholder and what are the implications for policies issued with standard mortality rates?',
 'Can the premium payment mode be changed after the policy has been issued, and what steps must be taken to make this change effective?',
 'Under what conditions can the Premium payment mode be changed, and when does this change take effect?',
 'What is the policy regarding the collection of advance Premiums, and how are these Premiums adjusted on the due date?',
 'What happens to the premium collected in advance on the due date, and does it carry any interest for the policyholder?',
 'Could you explain the options available for Premium Payment Term, Policy Term, and SA Multiple in case of a Single Pay under this insurance plan?',
 'Could you explain how the Safety Switch Option (SSO) works in the context of this investment plan and how it might benefit someone like Rahul, who is just starting his career?',
 "The context mentions eight different Unit Linked Funds with varying risk appetites. Could you elaborate on the characteristics of these funds and how one might choose between them, especially considering Rahul's situation?",
 'According to the given context, how much could Amit potentially save with a Cover Option 2 (Care Option) plan over a 15-year Premium Paying Term and Policy Term, assuming an annual gross investment return of 4% and 8%?',
 'Based on the example of Rahul, how did regular and disciplined contributions help him in creating a substantial corpus and developing a habit of savings towards a strong financial future?',
 'According to the provided context, what are the assumed annual gross investment returns for the maturity values shown in the table?',
 'Based on the given information, what would be the maturity value for a 100% investment in an Equity II Fund, assuming an annual gross investment return of 4% and an initial investment of 3,60,000?',
 'According to the provided context, what is the total maturity benefit for Mrs. Singh under the Cover Option 1 (Life Option) plan, assuming an 8% scenario?',
 'In the given illustrative example, if Amit were to unfortunately pass away at the end of the 5th policy year, what would be the total future premium funded by the company in the 4% scenario?',
 'How does the Maturity Benefit work for Mr. Rajpal under the Century Option, and what are the projected fund values at age 100 years with annual gross investment returns of 4% and 8%?',
 "In the event of Mr. Rajpal's death during the policy term, what kind of lump sum payment is made, and how does the company ensure that his savings for the desired goal continue uninterrupted?",
 'Based on the given data, how much would be the total maturity benefit for a 10-year investment with an annual premium of 20,00,000, assuming a gross investment return of 8%?',
 'According to the information provided, what is the lump sum paid on death for an individual who has paid an annual premium of 50,00,000?',
 'What are the total premiums paid and the corresponding maturity benefits for the assumed annual gross investment returns of 4% and 8% respectively, when Mr. Rajpal reaches the age of 100 years?',
 "In the event of Mr. Rajpal's unfortunate demise during the 15th policy year, what would be the death benefits based on the assumed annual gross investment returns of 4% and 8%, as shown in the illustrative examples?",
 'What is the minimum guaranteed death benefit percentage based on the premiums paid till the date of death under this Plan?',
 'Based on the given options, what are the different annual premium amounts available for the policyholder to choose from?',
 'What are the different cover options available in the plan and what benefits do they offer?',
 'How does the Premium Funding Benefit work and which charges are exempt from deduction in the unit account until maturity?',
 'How does the Premium Funding Benefit impact the policy after the death of the life assured in the given context?',
 'What are the three benefits under the CenturyOption Cover in the context, and how is the policy terminated based on these benefits?',
 'How is the Death Benefit calculated under the Cover Options mentioned in the policy, and who is eligible to receive it?',
 'What happens to the policy if a due premium is not paid after the completion of the Lock-in Period, according to the provided context?',
 'How is the Paid-up Sum Assured calculated in the given policy, and what factors does it depend on?',
 'Under what circumstances does the Premium Funding Benefit apply, and how are the funded premiums invested on behalf of the policyholder?',
 'What happens to the proceeds of the Discontinued Policy Fund if the policyholder passes away before the end of the Lock-in Period?',
 'How does the Policy terminate, according to the given context?',
 'How do the Loyalty Additions vary among the different fund options mentioned, and what role does the Policyholder play in determining the WealthBoosters proportion?',
 'Under the Care Option, what specific rights does the Claimant have after the death of the Life Assured regarding the Policy money at maturity?',
 'What limitations does the Claimant face regarding altering or opting out of fund-related options after the death of the Life Assured in the Care Option?',
 'How are withdrawals defined in the given context, and what are some examples of withdrawals mentioned in the text?',
 'What are the key differences between the "Life Option," "Care Option," and "Century Option" in terms of maturity benefits and policy termination?',
 'How can policyholders utilize the Settlement Option to receive their Maturity Benefit as a structured payout, and which Cover Options support this feature?',
 'How often are fund value related Loyalty Additions offered in this plan and what conditions must be met to receive them?',
 'Are Single Premium policies eligible for Loyalty Additions under this plan, and what other additional benefits are provided apart from Loyalty Additions?',
 'Under what circumstances are Loyalty Additions not applicable in the mentioned plan?',
 'How are Wealth Boosters calculated for each Unit Linked Fund in the described plan?',
 'Under what circumstances are "Wealth Boosters" post premium reduction credited in a similar manner according to the then prevailing Annualized Premium?',
 'For Regular or Limited Premium policies with a PPT greater than or equal to 10 years, what amount is specified when Emerging Leaders Equity Fund, India Multi Cap Equity Fund, Equity II Fund, Growth Plus Fund & Balanced Plus Fund are chosen?',
 'How does the annualized premium vary for a policy with a cover option 3 (Century Option) based on the chosen fund (Large Cap Advantage, Debt, or Liquid) and the number of years, as per the given table?',
 'In the illustrative example, for a policy with an annual premium of 5,00,000, how does the fund value before additions and the fund value after additions change over the years, and what role do policy admin charge and mortality charge play in this context?',
 'How are the Wealth Boosters calculated for Regular Premium policies under Cover Option 3 (Century Option) in each Unit Linked Fund, and how frequently are they paid after the completion of the first 30 Policy Years?',
 'What conditions apply to the Return of Mortality Charge feature, which adds the total of all Mortality Charges deducted during the Policy Term to the Fund Value at maturity, and what charges are excluded from this feature?',
 'Under what conditions is the Return of Mortality Charge feature excluded from the amount payable at maturity, and how is the remaining amount determined?',
 'In which specific policy scenarios, such as Single Premium Policies or certain statuses, is the Return of Mortality Charge not applicable?',
 'How is the Unit Price determined at the maturity date for the unit linked funds in this policy?',
 'In the given example, what are the loyalty additions, wealth boosters, and return of mortality charge applicable for Arun, who chose Equity II Fund with an annualized premium of Rs. 5,00,000 and a sum assured of Rs. 50,00,000 for Cover Option 1(Life Option)?',
 'Based on the given data, what is the total mortality charge for policy number 15 in the first year?',
 'Could you explain what the "Policy YearAnnual PremiumPolicy Admin Charge Mortality Charge Fund Value Before Additions Loyalty Additions Wealth Boosters Return of Mortality Charge Fund Value after Additions" table represents in the context of the provided document?',
 'What is the range of values for the "ChargeFund Value" before additions for the first 9 entries in the table?',
 'How does the "ChargeFund Value" after additions change from the 10th to the 15th entry in the table, and what are the additional charges and boosters during these years?',
 'What is the total fund growth rate mentioned at the end of the context, and how does it impact the presented figures?',
 'In line 15, there are several numerical values mentioned. Could you explain what each of these values (5,00,000, 91,70,188, 38,490, 2,88,679, and 94,97,357) represent in the context?',
 'What is the investment philosophy of the Emerging Leaders Equity Fund and what type of stocks does it primarily invest in?',
 'How does the Equity II Fund aim to generate long-term capital appreciation and what is its asset allocation range for high debt securities and money market & others?',
 'What is the primary investment focus of the Growth Plus Fund and what is the range of investment allocated to medium to high debt securities?',
 'How does the BalancedPlusFund aim to generate returns and what is the range of investment allocated to equities and medium debt securities?',
 'What is the primary objective of the Debt Fund (SFIN : ULI F00409/07/0 8INDE BTFU ND1 36) as mentioned in the context?',
 'Could you explain the investment and risk profile of the Large Cap Advantage Fund (SFIN: ULIF02109/06/20LARCPADFND136) as described in the provided context?',
 'What is the primary objective of the Large Cap Advantage Fund with the Segregated Fund Identification Number (SFIN) ULIF02109/06/20LARCPADFND136?',
 'In the scenario where the fund (SFIN: ULIF02109/06/20LARCPADFND136) attached to the Product (UIN: 136L064V03) does not comply with Regulation 8 of Schedule I of the IRDAI (Investment) Regulations, 2016, what option does the policyholder have?',
 'How are the units transferred from the Liquid Fund during each Policy Month, as per the given context?',
 'Could you explain how the Systematic Transfer Option (STO) helps an investor to mitigate the risk associated with market volatility and lump sum investment?',
 "How does the Systematic Transfer Option (STO) allow investors to enter the equity market, and what happens to the annual or single allocable premium once it's allocated to the Liquid Fund?",
 'In the context of STO, how are units transferred from the Source STO Fund to the Target STO Fund, and what restrictions are there on changing the Target STO Fund during its operation?',
 "Under what conditions can the 'Target STO Fund' option be availed in a Single Premium policy, and how is the switching of funds from the Liquid Fund to the 'Target STO Fund' carried out?",
 'What are the requirements and limitations for opting or re-opting for the STO in a policy where premiums are paid annually, and during which period is the STO operational in such cases?',
 'Under what specific conditions can the STO option be opted or re-opted, and when does it become non-operational?',
 'How can switching be performed among the Unit Linked Funds that are not STO Funds, and what happens to any remaining amount in those funds?',
 'Under what circumstances can STO not be exercised simultaneously with either RPO or AFR, and what are the exceptions to this rule?',
 'What are the four Unit Linked Funds that a policyholder can choose from, and what are their respective risk profiles?',
 'What is the purpose of mentioning funds with similar objectives and risk profiles in the context of the India Multi-Cap Equity Fund High, and how does it comply with Regulation 8 of Schedule I of the IRDAI (Investment) Regulations, 2016?',
 'How does the Company handle the situation when the proposed Large Cap Advantage Fund has the lowest FMC among its equity-oriented fund offerings, and how does it facilitate free switching to funds with similar risk profiles with their respective FMCs?',
 'What actions can lead to the cessation of the STO from the next Monthly Policy Anniversary, according to the given context?',
 'How does the Return Protector Option (RPO) work, and what role does it play in protecting gains from future equity market volatility?',
 'What are the specific conditions under which the RPO (Regular Premium Option) operates, and can the chosen RPO Fund be altered once it has been opted for?',
 'How does the investment process work under the RPO, and what range of fixed flat target appreciation percentage can be chosen by the investor?',
 'What is the range of fixed flat target appreciation percentages that can be chosen for the option mentioned in the context, and can this percentage be changed once the RPO is operational?',
 'Under what circumstances will the gain from the RPO Fund be transferred to the Debt Fund, and what happens if the gain from the RPO Fund is less than the Target Appreciation?',
 'Under what circumstances can RPO be exercised alongside other options such as STO, AFR, SSO, SWO, or MWO, and when does it cease to be operational in relation to these options?',
 'How does the cessation of RPO affect the investment of future premiums, and can RPO be re-opted after it has ceased during the Policy Term?',
 'Under what circumstances does the Reduced Pension Option (RPO) become invalid before the end of the Lock-in Period, and how can it be reinstated?',
 "Could you explain how the Auto Funds Rebalancing feature works, and provide an example of how a policyholder's savings allocation might be adjusted every three months using this feature?",
 'How frequently is the total Fund Value rebalanced in the Auto Funds Rebalancing feature, and what is the ratio distribution among the Equity II Fund, Balanced Plus Fund, and Debt Fund?',
 'Under what conditions can the Auto Funds Rebalancing (AFR) option be initiated, and are there any circumstances where the AFR would no longer be applicable during the Policy Term?',
 'According to the given context, how does the allocation of funds between "Other than Liquid Fund" and "Liquid Fund" change as the policy year progresses from T-3 to T?',
 'Under what circumstances can a request for AFR not be made, as mentioned in the provided context?',
 'Under what circumstances can a request for AFR not be made according to the Cover Option 2 (Care Option) of the policy?',
 'How does the Safety Switch Option (SSO) work in safeguarding funds from market volatility as the policy nears maturity?',
 'Under which specific conditions is the Safety Switch Option available in the given context, and what happens to the existing funds when it becomes operational?',
 'How does the rebalancing work for the amounts in the "Other than Liquid Fund" category when the Safety Switch Option is applied, according to the provided context?',
 'Under what circumstances would the Safety Switch Option (SSO) cease to be operational in relation to the Liquid Fund switches, as mentioned in the given context?',
 'How does a request for redirection of Premium impact the operational status of the Safety Switch Option (SSO), according to the provided context?',
 'Under what conditions can the SSO (Single Subscription Option) be activated and what happens to STO/RPO/AFR during the last four policy years?',
 'How does the Milestone Withdrawal Option (MWO) work, and what are the conditions for its availability and operation?',
 'Under what conditions can a policyholder switch or opt out of the Milestone Withdrawal Option or Systematic Withdrawal Option once the payouts have started?',
 'How is the MWO payout calculated, and what is the minimum requirement for the Fund Value after any withdrawal?',
 'Under what circumstances can the MWO option be re-activated after it has been stopped due to the Resultant Fund Value falling below the required threshold?',
 'Can the MWO feature be selected at any time during the policy term or only at specific times, such as policy inception or after a stipulated time under Cover Option 2 (Care Option)?',
 'Under what conditions can the MWO feature be re-activated or stopped, and how long does it take for the changes to come into effect?',
 'How does exercising the MWO option impact the functionality of the SSO or AFR options, and what happens to the remaining fund available in the policy account post MWO payout(s)?',
 'What are the age and policy year requirements for a policyholder to make partial withdrawals from this product without incurring any charges?',
 'What is the maximum amount that can be withdrawn partially from this product, and how does it relate to the annualized premium or single premium paid at the inception of the policy?',
 'Under what circumstances are partial withdrawals not allowed, particularly in relation to the Settlement Period, the RPO, and the death of the Life Assured with Cover Option 2 (Care Option) chosen?',
 'How does the Systematic Withdrawal Option (SWO) function in terms of partial withdrawals, and what percentage of the fund value is pre-decided to be withdrawn under this option?',
 'Under what conditions can a policyholder choose the Systematic Withdrawal Option (SWO) in their policy term?',
 'How does the frequency of withdrawals work under the Systematic Withdrawal Option (SWO), and what are the available modes for receiving these withdrawals?',
 'How is the frequency of withdrawals determined in relation to the chosen annual percentage for Systematic Withdrawal Option (SWO), and how is the withdrawal amount calculated based on the Fund Value?',
 'What is the minimum requirement for the Resultant Fund Value after any withdrawal, in relation to the Annualized Premium payable?',
 'Under what conditions would future withdrawals under the Systematic Withdrawal Option (SWO) be stopped, and how can this feature be re-activated?',
 'Can both the Milestone Withdrawal Option and the Systematic Withdrawal Option be exercised simultaneously within a single policy? If not, why?',
 "What restrictions are there on switching between the 'Milestone Withdrawal Option' and 'Systematic Withdrawal Option' in a policy, especially after the payout starts?",
 'How can the SWO (Systematic Withdrawal Option) percentage be changed during the policy term, and when does the change become effective?',
 'Under what conditions can the SWO (Systematic Withdrawal Option) payout be terminated and what happens to other features like SSO or AFR once SWO becomes operational?',
 'Can the SWO feature be exercised under all Cover Options, and are there any restrictions on when it can be requested under Cover Option 2 (Care Option)?',
 'What are the restrictions on exercising the Settlement Option in the policy, and are there any charges associated with it?',
 'How often can the policyholder receive payouts during the Settlement Period, and can this frequency be changed once the Settlement Option is operational?',
 'Once the Settlement Option becomes operational, can the frequency of payout during the Settlement Period be changed?',
 
 'How is the first installment calculated during the Settlement Period, and what happens to the investment risk during this period?',
 'What conditions must be met for switching to be allowed during the Settlement Period as mentioned in the context?',
 "According to the given context, what happens to the policy and what payment is made in the event of the life assured's death during the Settlement Period?",
 'What types of charges are deducted during the Settlement Period, and is it possible to make a request for a Settlement Option after the death of the Life Assured under Cover Option 2?',
 
 'Could you explain the Premium Redirection facility, including its cost, limitations, and the process for requesting it?',
 'Under what conditions can the Premium Redirection option be exercised and how does it affect other policy features like RPO, STO, AFR, and SSO?',
 'What are the rules and limitations for the Policyholder to decrease the premium payable under the policy after the first five completed Policy Years?',
 'Under what circumstances can the option to reduce the premium not be exercised according to the given context?',
 'What are the conditions specified for making a switch request under the given product rules?',
 'Under what conditions can a switch request be made, and are there any charges associated with it?',
 'How does selecting Cover Option 2 (Care Option) impact the ability to switch after the death of the Life Assured, and what happens to SSO or STO if a switch request is made?',
 'Under what circumstances is switching not allowed among the Unit Linked fund(s) other than STO Funds?',
 'What is the timeline and requirements for a Policyholder to change the premium payment mode, and are there any charges associated with this option?',
 'Under what circumstances can the policyholder surrender the policy and what is the process for calculating the surrender value, as per the Non-Forfeiture Provisions mentioned in the context?',
 'How is the state of the policy determined in case of discontinuance due to non-payment of premiums, and are there any exceptions to this rule according to the context provided?',
 'What is the minimum guaranteed interest rate applicable to the Discontinued Policy Fund and how is it determined?',
 'Under what circumstances, besides the death of the Life Assured, can the proceeds of the discontinued policies be paid by the insurer before the end of the lock-in period?',
 'How does the Discontinued Policy Fund operate in terms of crediting Fund Values and deducting Discontinuance Charges when a policy is surrendered or premium payment is discontinued during the Lock-in Period?',
 'What is the policy regarding the excess income earned in the Discontinued Policy Fund over the Minimum Guaranteed Interest Rate, and how is it apportioned in arriving at the proceeds of the discontinued policies?',
 'What happens to the risk cover and fund value if the due premium is not received by the expiry of the Grace Period during the first five years of the policy?',
 'If a policyholder decides to revive the policy but fails to do so within the Revival Period, how will the proceeds of the DPF be handled and what charges will be applicable during this period?',
 'What happens to the policy fund if the Policyholder chooses not to exercise their option, and how does the policy terminate in this scenario?',
 'What option does the Policyholder have regarding surrendering the Policy, and when are the proceeds of the discontinued policy payable?',
 'Under what circumstances can a policy be converted into a Reduced Paid-up Policy and what benefits will continue under this status?',
 'How is the Fund Value handled if a surrender request is received within the first five policy years and when will the policyholder receive the refunded proceeds?',
 'What happens to the Policy if the Premium is not received by the end of the Grace Period, and what options does the Policyholder have in this situation?',
 "If the Policyholder chooses to revive the Policy but fails to do so during the Revival Period, what happens to the Fund Value and the Policy's status?",
 'What happens to the policy if the policyholder does not exercise any of the given options and the policy continues to be in Reduced Paid-up status?',
 'Under what conditions and timeline can a policyholder surrender the policy and receive the Fund Value payable?',
 'What is the time frame within which one can apply for policy revival due to non-payment of premiums, and what are the conditions for such revival?',
 "How does the process of underwriting affect the revival of a discontinued policy during the lock-in period, and what is the significance of the company's approval in this context?",
 'When reviving a discontinued Policy, how does the company handle the collection of due and unpaid Premiums?',
 
 'What action does the company take regarding the discontinuance charges deducted at the time of policy discontinuance upon its revival?',
 'What are the different fund options available and what is the Fund Management Charge (p.a) for each of them in the given context?',
 'According to the context, what are the discontinuance charges for policies having an annualized premium up to Rs. 50,000 and above Rs. 50,000?',
 'What are the maximum limits for the lower of 10%*(AP or FV) and 3%*(AP or FV) subject in the given context?',
 'How is a discontinued policy revived and what are the terms and conditions regarding the collection of due premiums and levying of charges at the time of revival?',
 'What are the types of charges applied when a policy is surrendered or discontinued in this context, and are there any charges related to premium allocation and policy administration?',
 'How are fund management charges applied and deducted in the scenario presented, and what role does the NAV* play in their calculation?',
 'What are the maximum discontinuance charges for single premium policies above Rs. 3,00,000 for a policyholder aged 40?',
 "How are the annualized and single premium policies' discontinuance charges calculated, and what are the maximum limits for each policy type?",
 'How are the Mortality Charges calculated for this insurance plan, and how do they differ between male and female lives?',
 'Under Cover Options 1 & 3 during the policy term, how is the Sum at Risk (SAR) computed if the policy is in-force, and what factors are considered in this calculation?',
 'How is the Sum at Risk (SAR) calculated under Cover Options 1 & 3 during the policy term, considering the policy is in-force and the Sum Assured, withdrawals, and Fund Value?',
 'In the given context, what percentage of all Premiums paid is considered for calculating the Sum at Risk (SAR) if it is higher than the other two methods mentioned?',
 'Under what circumstances would the SAR (Surrendered Amount Ratio) be calculated as the higher of the Paid-up Sum Assured or 105% of all premiums paid for a Reduced Paid-up Policy?',
 'How is the SAR determined for Cover Option 2 during the Settlement Period when death occurs during the Policy Term?',
 'How does the Premium Funding Benefit Charge differ for female lives compared to male lives under Cover Option 2 in this policy?',
 'Are there any charges associated with partial withdrawals or switches in this policy, as mentioned in the given context?',
 'What are the charges associated with partial withdrawals and switches in this policy, and are there any additional miscellaneous charges to consider?',
 'How are the Net Asset Value (NAV) calculations performed for this policy, and which charges are guaranteed during the policy term?',
 'According to the current IRDAI guidelines, how is the Net Asset Value (NAV) of the fund calculated, and what components are included in this calculation?',
 'How frequently is the Net Asset Value (NAV) of the fund calculated, and what role does the valuation date play in this calculation?',
 'What is the policy regarding death due to suicide within the first 12 months of the policy or its revival, as per the provided context?',
 'According to the given context, how does the policy handle situations where the life assured is a minor?',
 'What happens to the policy when the Life Assured, who was a minor at the time of policy commencement, reaches the age of 18?',
 "In the case of the Life Assured being a minor, who receives the policy proceeds in the event of the Life Assured's death and what happens to the policy subsequently?",
 'What is the duration of the free-look period for policyholders who received their policy document through the Distance Marketing Mode?',
 'If a policyholder decides to cancel their policy during the free-look period, what costs will be deducted from the refunded fund value?',
 'What is the duration of the grace period for annual, half yearly, and quarterly modes of premium payment, and how does it differ from the monthly mode in terms of the continuation of life insurance cover?',
 
 'Under what conditions can an Invest4G policy terminate automatically during the policy term, and what happens to the remaining fund value upon such termination, especially considering the Cover Option 2 (Care Option)?',
 'How is the first premium allocated in this policy and when does it take effect?',
 'What are the guidelines for transaction requests, such as renewal premiums or partial withdrawals, in terms of cut-off time and NAV allocation, as per the context provided?',
 'What are the prohibited actions related to rebates under Section 45 of the Insurance Act, 1938 as amended from time to time, and what is the penalty for non-compliance?',
 'How does Section 41 of the Insurance Act, 1938 as amended from time to time, regulate the offer and acceptance of rebates in relation to insurance policies in India?',
 'How does the Canara HSBC Life Insurance Company Limited handle cases of fraud and misrepresentation, and where can one find the complete provisions regarding this matter?',

 "What is the shareholding pattern of the joint venture that forms the Canara HSBC Life Insurance Company Limited, and what is the company's primary aim in providing a range of life insurance products?",
 'What is the main objective of Canara HSBC Life Insurance Company Limited in providing a range of Life Insurance products?',
 'How does the SFIN (Segregated Fund Index Number) for Large Cap relate to the funds offered under the Canara HSBC Life Insurance Invest 4G contract?',
 'What are the SFIN numbers for the Large Cap Advantage Fund and the Debt Fund mentioned in the contract, and what do these numbers signify?',
 'Could you explain the difference between Unit Linked Insurance products and traditional insurance products as mentioned in the given context, and what are the potential risk factors associated with Unit Linked Insurance products?',
 'How are Unit Linked Insurance products different from traditional insurance products in terms of risk factors?',
 'How can a policyholder check the value of their policy wise units in the Canara HSBC Life Insurance Invest 4G plan?',
 'What are the contact details provided for Canara HSBC Life Insurance Company Limited, including their registered office address, head office address, phone numbers, email, and website?',
 'Which company is the insurance products offered and underwritten by, as mentioned in the context, and where is its head office located?',
 'Where is the head office of the Insurer, who is underwriting the policy, located and what is its registration number?',
 'What action is recommended for individuals who receive spurious or fraudulent phone calls regarding insurance policies, bonuses, or investments?']

In [25]:
metadata=[]
for que in questions:
    sim_docs = vectorstore.similarity_search(que,k=3)
    metadata.append(sim_docs)

for i in metadata:
    print(i)

[Document(metadata={'page': 24, 'source': 'invest4g_Product_Brochure.pdf'}, page_content="8. Switches Charges: No Charge.\n9. Miscellaneous Charges: No Charge\nAll charges are exclusive of Goods and Services Tax & applicable cess (es)/levy, if any, as applicable and amended from time \nto time which will be borne by the Policyholder. In case of death of Life Assured under Cover Option 2 (Care Option), all \ncharges except Mortality & Premium Funding Beneﬁt Charges will continue to be deducted during Policy Term. All the charges mentioned above except Fund Management Charge will be deducted through cancellation of units. The Fund Management Charges (except for Large Cap  Advantage Fund, Debt Fund & Liquid Fund), Mortality Charges and Premium Funding Beneﬁt Charges mentioned above are guaranteed during the Policy Term.\n*Net Asset Value (NAV) calculation: NAV shall be calculated on all Business Days in accordance with the IRDAI's guidelines in \nforce from time to time. As per the presen

In [25]:
import time
results = []
for question in questions:
    result = rag_chain.invoke({"input": question})
    results.append(result["answer"])
    time.sleep(2)


In [27]:
for result in results:
    print(result)
    print("*"*50)
    print()

Other charges that might be deducted are Switch Charges, Miscellaneous Charges, Surrender/Discontinuance Charges, and Premium Funding Benefit Charges. Switch Charges and Miscellaneous Charges are currently not applicable as per the given context. Surrender/Discontinuance Charges are levied on the Fund Value upon surrender or discontinuance of the policy. Premium Funding Benefit Charges are deducted to maintain savings contributions in the policyholder's absence.
**************************************************

The policyholder cannot surrender or withdraw monies invested in Unit Linked Insurance Products during the first five years. This is because these products do not offer any liquidity during this period. Therefore, surrender or withdrawal is not allowed until the end of the fifth year.
**************************************************

The three options available under the Invest 4G plan offered by Canara HSBC Life Insurance are Life Option, Care Option, and Century Option. Th

In [28]:
results

["Other charges that might be deducted are Switch Charges, Miscellaneous Charges, Surrender/Discontinuance Charges, and Premium Funding Benefit Charges. Switch Charges and Miscellaneous Charges are currently not applicable as per the given context. Surrender/Discontinuance Charges are levied on the Fund Value upon surrender or discontinuance of the policy. Premium Funding Benefit Charges are deducted to maintain savings contributions in the policyholder's absence.",
 'The policyholder cannot surrender or withdraw monies invested in Unit Linked Insurance Products during the first five years. This is because these products do not offer any liquidity during this period. Therefore, surrender or withdrawal is not allowed until the end of the fifth year.',
 'The three options available under the Invest 4G plan offered by Canara HSBC Life Insurance are Life Option, Care Option, and Century Option. These options ensure that your life is fully secured and cater to different life stages.',
 'The

In [40]:
import time
results = []
for question in questions:
    result = rag_chain1.invoke({"input": question})
    results.append(result["answer"])
    #time.sleep(2)

RemoteProtocolError: Server disconnected without sending a response.

In [42]:
len(results)

66

In [1]:
questions=['What are the charges that will be deducted from the policy, according to the context?',
 'Is there an option for a refund of mortality charges in all three options of the Invest 4G plan?',
 'When do the additions start in the policy, and what are the conditions for their availability?',
 'What is the policy regarding liquidity and surrender/withdrawal of monies invested in Unit Linked Insurance Products during the first five years of the contract?',
 'Who bears the investment risk in the investment portfolio of this policy, according to the context?',
 'What type of insurance plan is the Canara HSBC Life Insurance Invest 4G and what are its key features?',
 "How does the Canara HSBC Life Insurance Invest 4G plan provide financial protection for loved ones in case of the policyholder's demise?",
 'What are the different cover options available in the Canara HSBC Life Insurance Invest 4G plan and which life stages do they cater to?',
 'How does the Canara HSBC Life Insurance Invest 4G plan maximize savings opportunities for policyholders with its low charges structure?',
 'What is the Premium Funding Benefit offered by the Canara HSBC Life Insurance Invest 4G plan?',
 'What are the intervals for receiving loyalty additions in this policy and how long do they continue?',
 'How often are wealth boosters provided and for what duration during the policy term?',
 "What are the options available for managing the policy portfolio to optimize returns according to one's investment preference?",
 'Which tax benefits can be availed on premiums paid and benefits received under this policy, as per the Income Tax Act, 1961?',
 'What types of charges are included in the risk charges of this policy, and when are premium funding benefit (PFB) charges applicable?',
 'What are the different cover options available under the plan and what is the eligibility criteria for each option?',
 'How does the "Life Option" cover differ from the "Care Option" cover in terms of benefits and payouts?',
 'What is the "Premium Funding Benefit" mentioned under the "Care Option" cover and how does it work?',
 'What happens to the policy if the life assured survives till the maturity of the policy under the "Life Option" cover?',
 'What is the maximum policy term available for each cover option and what are the entry age requirements for the policy?',
 'What are the different types of withdrawals mentioned in the given context and when are they typically made?',
 'Could you explain the Century Option cover and its benefits for the Life Assured and their loved ones?',
 'What is the death benefit payable under the Century Option, and how is it calculated?',
 'What happens to the policy if the Life Assured survives until they turn 100 years old?',
 'Can the chosen Cover Option be changed at any point during the Policy Term, or is it fixed from the inception stage?',
 'What are the different premium payment modes available in the plan?',
 'How is the annualized premium calculated in the policy?',
 'What is the definition of age used in the plan and what are the entry ages applicable for policies issued with standard mortality rates?',
 'Can the policyholder change the premium payment mode during the policy term and if so, what is the process for doing so?',
 'What is the maximum single premium amount that can be paid at the inception of the policy?',
 'What changes are not allowed in the Premium Payment Mode after the death of the Life Assured if the Care Option is chosen?',
 'How many months in advance can the company accept the first premium payments for monthly payment policies at the policy inception?',
 'What is the maximum period for collecting premiums due in a financial year in advance, if they are being collected in the previous financial year?',
 'What options does the policyholder have in terms of choosing their Premium Payment Term and Policy Term?',
 'In case of a Single Pay option, what choices are available for the SA Multiple?',
 'What are the four different Portfolio Management Options available in the mentioned plan, and how do they help manage and build wealth?',
 'In the illustrative example of Rahul, what is the monthly premium he chose to start saving with, and what is the Premium Paying Term & Policy Term for his plan?',
 "What are the maturity values for Amit's plan under multiple scenarios, assuming an annual gross investment return of 4% and 8% with 100% investment in an Equity II Fund?",
 'What are the four different cover options available in the plan, and how do they cater to different life stages and financial goals?',
 'Can you explain the concept of the Systematic Transfer Option (STO) and the Return Protector Option (RPO) mentioned in the context, and how they might benefit an investor?',
 "What are the different investment return scenarios presented for Mrs. Singh's Cover Option 1 plan, and what is the corresponding fund value at the end of the policy term?",
 "In the case of Illustrative Example 3, how does the plan ensure that Rahul's savings for his desired goal continue uninterrupted in the event of his death during the policy term?",
 'How does the Century Option plan chosen by Mr. Rajpal differ from the Cover Option 1 plan chosen by Mrs. Singh in terms of premium payment period and policy term?',
 'What is the death benefit for Amit, as described in the context, and how does it compare to his maturity benefit in terms of annual premium and total maturity benefit?',
 "What is the purpose of the lump sum benefit provided in Mrs. Singh's Cover Option 1 plan in the event of her death during the policy term?",
 'What is the total amount of premiums paid as mentioned in the context?',
 'Can you explain the pattern of the numbers listed in the first two lines of the page content?',
 'How many different amounts are listed for lump sum paid on death in the provided context?',
 "What is the significance of the numbers '1', '2', and '5' that appear multiple times in the first two lines?",
 'Based on the context, what can be inferred about the purpose or nature of this document?',
 'What are the different premium payment options and their corresponding sum assured totals as per the given table?',
 'How do the maturity benefits vary based on the assumed annual gross investment returns of 4% and 8%?',
 'What is the guaranteed death benefit mentioned in the context, and how does it impact the overall death benefit calculation?',
 'How are the fund values calculated, and what charges are included in the deductions mentioned in the context?',
 "Can you explain the significance of the table presenting the death benefit in case of Mr. Rajpal's unfortunate death in the 15th policy year?",
 'What are the different Cover Options available in the plan and what benefits do they offer?',
 'How is the Death Benefit calculated under each Cover Option, and does it take withdrawals into account?',
 'What happens to the policy after the payment of any of the benefits mentioned under each Cover Option?',
 'What is the Premium Funding Benefit mentioned under Cover Option 2, and how does it affect the charges deducted from the unit account?',
 'How does the policy ensure that the Sum Assured payable on death is not reduced by withdrawals made prior to the death of the Life Assured under Cover Option 2?',
 'What are the different cover options available for the Death Benefit, and how is it determined based on the chosen option?',
 'Who is considered as the Claimant for the purpose of payment of death benefits, as per the provided context?',
 'Can you explain what happens to the policy if the due premium is not paid after the completion of the Lock-in Period?',
 'How is the Paid-up Sum Assured calculated, as per the given context?',
 'What is the Premium Funding Benefit, and how does it work in case of the death of the Life Assured?',
 'What are the different Cover Options available in the policy and what benefits are payable under each option at maturity?',
 'How do Loyalty Additions work in the Unit Linked Fund and what percentage is allocated to each fund type?',
 'What happens to the Premium Funding Benefit if the policy enters a Discontinuance or Reduced Paid-up state?',
 'What rights does the Claimant have under the Care Option in terms of altering fund-related options after the death of the Life Assured?',
 'How are withdrawals defined in the policy and what are the different types of withdrawals mentioned in the context?',
 'What are the benefits of the Century Option if the Life Assured survives until the policy maturity?',
 'How can the policyholder receive the Maturity Benefit under the Settlement Option?',
 'What regular additions does the product offer, and how are they calculated?',
 'At what intervals are Loyalty Additions provided, and are they applicable for Single Premium policies?',
 'What are Wealth Boosters, and how are they determined for each Unit Linked Fund?',
 'What happens to the Wealth Boosters post premium reduction under the Linked Fund, if the Premium Reduction option is exercised?',
 'How are the Wealth Boosters credited for Limited Premium policies with PPT less than 10 years when Emerging Leaders Equity Fund, India Multi Cap Equity Fund, Equity II Fund, Growth Plus Fund & Balanced Plus Fund are chosen?',
 'How are the Wealth Boosters credited for Limited Premium policies with PPT less than 10 years when Large Cap Advantage Fund, Debt Fund & Liquid Fund are chosen?',
 'For Regular/Limited Premium policies with PPT greater than or equal to 10 years, how are the Wealth Boosters credited when Emerging Leaders Equity Fund, India Multi Cap Equity Fund, Equity II Fund, Growth Plus Fund & Balanced Plus Fund are chosen?',
 'What is the maximum amount of Wealth Boosters that can be credited for Regular/Limited Premium policies with PPT greater than or equal to 10 years when Emerging Leaders Equity Fund, India Multi Cap Equity Fund, Equity II Fund, Growth Plus Fund & Balanced Plus Fund are chosen?',
 'What are the different annualized premium options available for the Large Cap Advantage Fund, Debt Fund, and Liquid Fund, and what are their respective charges for policy years 10, 20, and 30?',
 'How are the Wealth Boosters calculated for Regular Premium policies under Cover Option 3 (Century Option) after the completion of the first 30 policy years?',
 'What conditions must be met for the Return of Mortality Charge to be added to the Fund Value at the maturity date?',
 'In the illustrative example provided, what are the Policy Admin Charges for the policy years 5, 10, and 15, and what are the corresponding Fund Values before any additions?',
 'How are the Mortality Charges excluded from the amount payable under the Return of Mortality Charge feature for sub-standard lives due to health or higher risk factors other than health?',
 'What components are excluded from the amount payable under the Return of Mortality Charge feature?',
 'How is the amount of Return of Mortality Charge determined in relation to the value of total units held in the unit linked funds at maturity?',
 'Under what circumstances is the Return of Mortality Charge not applicable, according to the given context?',
 'If Arun chooses to pay an annualized premium of Rs. 5,00,000 for Cover Option 1 with a policy term of 20 years, what are the Loyalty Additions, Wealth Boosters, and Return of Mortality Charge that may be applicable to him?',
 "According to the provided example, what is the fund growth rate assumed for Arun's policy, and what are the ranges of values for the given categories?",
 'What is the range of the mortality charge for the policy in the first year?',
 'What is the fund value before additions for the policy in the tenth year?',
 'What is the policy admin charge for the policy in the fifth year?',
 'What is the loyalty addition for the policy in the tenth year?',
 'What is the return of mortality charge fund value for the policy in the fifteenth year?',
 'What is the investment philosophy of the Equity II Fund and what is the range of asset allocation for equity investments in this fund?',
 'How does the BalancedPlusFund aim to generate returns for its investors, and what is the range of asset allocation for medium debt securities in this fund?',
 'What is the primary investment objective of the Liquid Fund, and what is the maximum percentage allocated to low debt securities in this fund?',
 'How does the India Multi-Cap Equity Fund aim to generate capital appreciation, and what is the range of asset allocation for small cap, mid cap, and large cap companies in this fund?',
 'What are the primary investment objectives of the Growth Plus Fund, and what is the range of asset allocation for medium to high debt securities in this fund?',
 'What is the primary objective of the Liquid Fund mentioned in the context, and what risk and liquidity levels are associated with it?',
 'How does the described Plan offer flexibility to manage and control savings? Can you elaborate on the available Unit Linked Fund options and their allocation?',
 'What is the main investment goal of the Large Cap Advantage Fund, and what type of stocks does it primarily invest in?',
 'What specific Segregated Fund Identification Number (SFIN) and Product Name are mentioned in the context, and how do they relate to the IRDAI (Investment) Regulations, 2016?',
 'In the event of non-compliance with the mentioned regulations, what option is provided to the policyholder for fund switching?',
 'What is the purpose of the Systematic Transfer Option (STO) in this context?',
 'How are units transferred from the Liquid Fund to the Target STO Fund on a monthly basis?',
 "Can the 'Target STO Fund' be changed while the STO is operational?",
 'How long is the STO operational for a Single Premium policy?',
 'What are the conditions for availing the STO option?',
 'Under what conditions can the STO option be availed or re-availed, and when does it become effective?',
 "What are the limitations on opting for STO when it comes to premium payment modes and the policy's premium payment term?",
 'How does the investment of remaining amounts work in other funds when STO is opted, and what are the switching options available among Unit Linked Funds?',
 'Can STO be opted out at any time during the premium payment term, and when will it become effective?',
 'Are there any restrictions on exercising STO simultaneously with other options like RPO, AFR, SSO, SWO, or MWO, and what happens when these options become operational?',
 'What specific regulation from the IRDAI (Investment) Regulations, 2016 is being referred to in this context?',
 "What is the nature of the proposed fund discussed in the context, and why can't the company offer a lower FMC for it?",
 'How will the company accommodate investors who wish to switch to a fund with a similar risk profile as the proposed fund?',
 'What types of equity-related securities can be invested in, according to the context?',
 'Apart from equity-related securities, what other types of investments are mentioned in the context?',
 'What are the specific requests that can lead to the cessation of the STO from the next Monthly Policy Anniversary?',
 'How are future premiums invested once the STO ceases, and can this be specified otherwise?',
 'What is the Return Protector Option (RPO), and how does it protect gains from future equity market volatility?',
 'What are the conditions for the RPO, specifically regarding the choice of RPO Fund and Target Appreciation?',
 'Can the RPO be chosen at any time during the policy term, or is there a specific time frame for its activation?',
 'What is the range of the Target Appreciation that can be chosen for the RPO and when can this option be selected?',
 'How does the Fund Value in the RPO Fund get transferred to the Debt Fund and what happens if the gain from the RPO Fund is less than the Target Appreciation?',
 'Under what circumstances can RPO be exercised simultaneously with other options like STO, AFR, SSO, SWO, and MWO?',
 'What happens to the RPO when the policyholder requests for Premium redirection, switching, or partial withdrawals?',
 'How does the RPO behave once it ceases to exist or is opted out, and what happens to future Premiums?',
 'What is the purpose of the Auto Funds Rebalancing (AFR) feature mentioned in the context?',
 'How often does the Auto Funds Rebalancing option rebalance the allocation of savings in various Unit Linked Funds?',
 'Can the Auto Funds Rebalancing option be activated at any time during the Policy Term, or is it limited to specific periods?',
 'What happens to the Auto Funds Rebalancing feature if a switch or premium redirection is exercised?',
 'In the given example, how is the total Fund Value rebalanced every 3 months using the Auto Funds Rebalancing option?',
 "What are the different fund allocations in the 'Safety Switch Option' during the last four years of the policy term?",
 "How does the 'Automatic Fund Rebalancing' (AFR) interact with other options like STO, RPO, and SSO?",
 'Under what circumstances can a request for AFR not be made?',
 'What happens to the AFR when partial withdrawals or SWO/MWO are made while it is operational?',
 'How does the company rebalance the funds when the Safety Switch Option becomes operational?',
 'Under which specific cover options is the Safety Switch Option (SSO) available, and are there any restrictions on when a request for SSO can be made under Cover Option 2 (Care Option)?',
 'What are the charges associated with auto-rebalancing units from "Other than Liquid Funds" to "Liquid Fund" when SSO is operational?',
 'How does switching in or out of the Liquid Fund impact the operation of the SSO, according to the given context?',
 'In the scenario where SSO is operational, how is it affected by a partial withdrawal or the operation of SWO/MWO? What happens to the balance of the Fund Value in these cases?',
 'If a request for redirection of premium is made while SSO is operational, what is the impact on the effectiveness of the SSO, as per the provided information?',
 'Under what conditions can the Systematic Partial Withdrawal Facility (SSO) be opted for in the plan? What happens to STO/RPO/AFR during the last four policy years?',
 'How does the Milestone Withdrawal Option (MWO) work, and when is it available for the policyholder to receive a payout?',
 'What happens to the units in the funds when the MWO payout is made, and how is the unit price determined for each MWO payout?',
 'Can a policyholder choose both the Milestone Withdrawal Option and the Systematic Withdrawal Option in the same policy? What happens if they try to switch options after the payout has started?',
 'What is the minimum Fund Value requirement after any withdrawal from the policy, and how is it calculated?',
 'What is the minimum resultant fund value required after any withdrawal for Regular/Limited Premium policies and Single Premium policies?',
 'Can the MWO feature be re-activated after it has been stopped, and if so, what is the process for doing so?',
 'How does the MWO feature interact with the SSO or AFR options, and what happens to these options after an MWO payout?',
 'In what circumstances can the MWO feature not be exercised, and what is the settlement period mentioned in this context?',
 'What are the conditions for making partial withdrawals from this product, and what is the minimum withdrawal amount allowed?',
 'What conditions must be met for the benefits under the policy to be applicable?',
 'What is the minimum amount that can be withdrawn partially from the policy?',
 'How many partial withdrawals are allowed to be made in a policy year?',
 'What is the maximum partial withdrawal amount allowed in case of Regular/Limited Premium policies?',
 'In case of Single Premium policies, what should be the minimum fund value immediately after a partial withdrawal?',
 'Under what conditions can the Systematic Withdrawal Option (SWO) be chosen and what is the range of withdrawal percentages available for the Policyholder?',
 'How do partial withdrawals affect the operation of SSO, AFR, and STO in a policy account, and what happens when a partial withdrawal request is made from the Liquid Fund while STO is operational?',
 'Are partial withdrawals allowed during the Settlement Period or when the RPO is operational, and what is the impact of partial withdrawals on Sum Assured payable on death under Cover Option 2 (Care Option)?',
 'What are the eligibility criteria for the Policyholder to choose the SWO option and what are the payment modes available for withdrawals under this option?',
 'How does the operation of partial withdrawals differ when SSO or AFR is operational compared to when STO is operational?',
 'What are the different modes in which the withdrawals can be payable under the Systematic Withdrawal Option (SWO)?',
 'How is the annual percentage chosen for SWO split across a Policy Year depending on the chosen frequency? Could you provide an example?',
 'What are the conditions that may trigger the stopping of future withdrawals under the SWO feature?',
 "Can the Policyholder switch between 'Milestone Withdrawal Option' and 'Systematic Withdrawal Option' once the payout from either of the options has started? Why or why not?",
 'How often can the SWO percentage be changed during the Policy Term, and can it be changed even after SWO has become operational?',
 'Under what circumstances can the SWO percentage be changed during the Policy Term?',
 'How is the SWO payout calculated and redeemed from the funds?',
 'Can the SWO feature be exercised if the SSO or AFR is already operational? How will it affect these options?',
 'What happens to the RPO/STO if the SWO feature becomes operational?',
 'Can the SWO feature be opted for when the Policy is in a Reduced Paid-up state?',
 'What are the restrictions on exercising the Settlement Option in the policy?',
 "How does the frequency of payout during the Settlement Period impact the policyholder's benefits?",
 'What role does the policyholder continue to play in managing investment risks during the Settlement Period?',
 'How is the risk cover maintained during the Settlement Period, especially when the Care Option is chosen?',
 'Are there any charges associated with exercising the Settlement Option in the policy?',
 'What happens to the risk cover and mortality charges when death occurs during the policy term under the Care Option?',
 'If the life assured passes away during the settlement period, what amount will be payable, and what happens to the policy?',
 'Are partial withdrawals, SWO, MWO, RPO, SSO, and AFR allowed during the settlement period?',
 'What charges continue to be deducted during the settlement period, and are there any other charges applicable?',
 'Can the policyholder request full withdrawal to close the account during the settlement period, and what happens to the policy and remaining fund value in such a case?',
 'What activity can be performed without impacting SSO in this context?',
 'After how many completed policy years does the policyholder have the option to decrease the premium?',
 'What is the maximum percentage by which the policyholder can decrease the premium under the policy?',
 'Are there any minimum premium limits applicable to the premium reduction option, and if so, how are they determined?',
 'What conditions must be met for the policyholder to be eligible for a premium reduction?',
 'What happens to the Sum Assured under the policy when the premium is reduced, and are there any conditions under which this option cannot be exercised?',
 'How many times can the Policyholder switch the investment pattern during the Policy Term or Settlement Period, and are there any charges associated with this action?',
 'If Regular Premium Option (RPO) or Auto Fund Rebalancing (AFR) is operational, how does a request for switching impact these options?',
 'How does switching into or out of the Liquid Fund impact the Systematic Switching Option (SSO), and what are the conditions for switching while SSO is operational?',
 'Can the Policyholder change the premium payment mode, and what steps must be taken to do so?',
 'How can the policyholder change the premium payment mode under this policy?',
 'When will the change in premium payment mode become effective after a written request is submitted?',
 'Are there any charges for exercising the option to change the premium payment mode?',
 'How far in advance should the policyholder request a change in premium payment mode for it to be effective from the next policy anniversary?',
 'Is it possible to change the premium payment mode post the death of the life assured if Cover Option 2 (Care Option) is chosen?',
 'What happens if the policyholder decides to surrender the policy, and is there a difference in this process under Cover Option 2?',
 'How is the state of the policy defined when it comes to discontinuance, and what are the exceptions to this definition?',
 'What is the minimum guaranteed interest rate applicable to the Discontinued Policy Fund, and how is it determined?',
 'Could you explain the concept of a lock-in period in the context of this policy, and what are its implications for the policyholder?',
 'What is the Discontinued Policy Fund (DPF), and how does it relate to policies that are surrendered or discontinued?',
 'What is the purpose of the Discontinued Policy Fund (DPF) as mentioned in the policy terms?',
 'How are the amounts credited to the DPF earning interest, and what happens to the excess income earned?',
 'What are the consequences for a policyholder if they discontinue their policy during the lock-in period?',
 'If a policyholder does not revive their policy during the Revival Period after it has been transferred to the DPF, when will the proceeds of the DPF be paid out to them?',
 'What charges are applicable to the DPF during the period when a policy is in the DPF but not revived by the policyholder, and the revival period ends after the lock-in period?',
 'What happens to the policy if the policyholder does not exercise the specified option?',
 'How will the policy fund be invested if the policyholder does not opt for the risk cover?',
 'When will the proceeds of the DPF be paid to the policyholder?',
 'What is the consequence of the policyholder surrendering the policy at any time?',
 'Under what circumstances will the proceeds of the discontinued policy be payable?',
 'What happens to the policy benefits and charges during the Grace Period mentioned in the context?',
 'How is the Fund Value transferred and the proceeds refunded if a surrender request is received within the first five policy years?',
 'How is a policy converted into a Reduced Paid-up Policy after the Lock-in Period, and what options does the Policyholder have in such a scenario?',
 'What are the potential outcomes if the Policyholder opts to revive the Policy but does not do so within the Revival Period?',
 'Can the Policyholder surrender the Policy at any time, and how is the Fund Value payable in such a case?',
 'What happens to the policy benefits and charges during the Grace Period?',
 'Under what conditions can a policyholder receive the Fund Value after submitting a surrender request?',
 'What is the duration of the Revival Period, and what actions must be taken to revive a discontinued policy within this period?',
 'What are the requirements for reviving a discontinued policy during the Lock-in Period, specifically regarding the collection of due and unpaid premiums and the levying of charges?',
 'How is the risk cover and investments in Unit Linked Funds handled when reviving a discontinued policy during the Lock-in Period?',
 'What is the current Fund Management Charge (FMC) for the Discontinued Policy Fund?',
 'How does the FMC on Debt Fund, Large Cap Advantage Fund, and Liquid Fund change, and who needs to approve this change?',
 'What charges are levied at the time of revival of a Reduced Paid-up Policy?',
 'What are the Surrender/Discontinuance Charges for Regular/Limited Premium policies?',
 'What are the policy administration charges for Regular/Limited Premium policies?',
 'What charges are applicable for Regular or Limited Premium policies in terms of Surrender or Discontinuance?',
 'Are there any Premium Allocation Charges for Regular or Limited Premium policies?',
 'Does the policy have any Policy Administration Charges for Regular or Limited Premium policies?',
 'How are the Fund Management Charges applied and calculated for Regular or Limited Premium policies?',
 'What is the significance of the NAV* mentioned in the context of Fund Management Charges for Regular or Limited Premium policies?',
 'What are the discontinuance charges for policies with a single premium above Rs. 3,00,000 for the 3rd year, and how are they calculated based on the Single Premium or Fund Value?',
 'How is the Sum at Risk (SAR) calculated during the policy term under Cover Options 1 & 3, and what factors determine the higher value in the computation?',
 'What is the mortality charge rate for a female life assured aged 40, and how does it compare to the rate for a male of the same age?',
 'For a policy with an annualized premium, what is the maximum charge that can be levied as a surrender charge in the first year?',
 'Can you explain the difference in the calculation of the Sum at Risk (SAR) for Cover Options 1 & 3 versus Cover Options 2 & 4 during the policy term?',
 'How is the SAR computed for a Reduced Paid-up Policy under Cover Options 1 and how does it differ for Cover Option 2?',
 'What types of withdrawals are included in the calculation of SAR for a policy?',
 'How does the Premium Funding Benefit Charge differ between male and female lives under Cover Option 2?',
 'Are there any charges for partial withdrawals or switches made on a policy?',
 'Under what circumstances are no Premium Funding Benefit Charges applied to a policy?',
 'What charges are the policyholder responsible for, according to the provided context?',
 'How are the charges mentioned in the context deducted?',
 'Which charges are guaranteed during the policy term, as per the context?',
 "How is the Net Asset Value (NAV) calculated, according to the IRDAI's guidelines mentioned in the context?",
 'In the case of death under Cover Option 2, which charges continue to be deducted during the policy term?',
 'What happens to the policy if the life assured dies due to suicide within the first 12 months of the policy or its revival?',
 'How is the benefit amount calculated in case of death due to suicide within the first 12 months of the policy or its revival?',
 "What is the policy's provision for a minor life assured and how does the policyholder change once the life assured turns 18?",
 'What happens to the policy if the life assured, who is a minor, dies before turning 18?',
 'Can the policyholder cancel the policy within a certain period if they disagree with its terms and conditions, and how long is this period?',
 "What happens to the Policyholder's Fund Value if they choose to cancel the Policy during the free-look period?",
 'How long is the grace period for monthly premium payments, and what happens during this period regarding life insurance cover?',
 'Under what circumstances can the Policy terminate automatically, and what happens to the remaining Fund Value in such cases?',
 'How are transaction requests, such as renewal premiums and partial withdrawals, allocated in terms of NAV based on the cutoff time?',
 'Are there any tax benefits under the Policy, and who should be contacted for tax-related queries?',
 'Is there an option for taking a loan under this plan?',
 'What are the tax benefits available under this policy and how are they subject to change?',
 'Who should be contacted for tax-related queries regarding this plan?',
 'Are assignments allowed under this plan, and if so, what is the relevant section of the Insurance Act?',
 'Is nomination permitted under this plan and which section of the Insurance Act governs it?',
 'What is the prohibition outlined in Section 41 of the Insurance Act, 1938 regarding rebates and what is the penalty for non-compliance?',
 'How does Canara HSBC Life Insurance Company Limited handle fraud and misrepresentation, as per the context provided?',
 'What is the shareholding pattern of the joint venture between Canara Bank, Punjab National Bank, and HSBC Insurance (Asia Pacific) Holdings Limited in the formation of Canara HSBC Life Insurance Company Limited?',
 'What is the aim of Canara HSBC Life Insurance Company Limited in providing life insurance products to its customers?',
 'What is the significance of the names "Canara HSBC Life Insurance Company Limited" and "Canara HSBC Life Insurance Invest 4G" in relation to the quality, future prospects, or returns of the insurance contract?',
 "What does the name 'Insurance Invest 4G' signify about the quality, future prospects, or returns of the unit linked insurance contract?",
 "Could you list the SFIN (Segregated Fund Index Number) for the 'Emerging Leaders Equity Fund' and 'Debt Fund' offered under the 'Insurance Invest 4G' contract?",
 'How are Unit Linked Insurance products different from traditional insurance products, and what risk factors are they subject to?',
 'What happens to the premium paid in Unit Linked insurance policies in relation to the investment risks associated with capital markets and NAVs of the units?',
 "Where can the policyholder find the value of policy-wise units as per the FORM D02 for the 'Canara HSBC Life Insurance Invest 4G' plan?",
 'What is the registered office address of Canara HSBC Life Insurance Company Limited?',
 'What are the contact numbers provided for Canara HSBC Life Insurance Company Limited?',
 'What is the UIN number and BRONJIN code mentioned in the context?',
 'According to the context, what is the role of IRDAI in insurance activities?',
 'What should the public do if they receive spurious/fraud phone calls related to insurance activities?',
 'What are the charges that will be deducted from the policy?',
 "Under what conditions are 'Additions' available in the policy?",
 "Why can't the policyholder surrender or withdraw the monies invested in Unit Linked Insurance Products for the first five years?",
 'What are the three options available under the Invest 4G plan?',
 'Who bears the investment risk in the investment portfolio of this policy?',
 'What are the three cover options offered by the Canara HSBC Life Insurance Invest 4G plan?',
 "How does the plan ensure that savings contributions are made even in the policyholder's absence?",
 'What type of product is the Canara HSBC Life Insurance Invest 4G plan?',
 'When are Loyalty Additions made to the fund, and how often do they occur?',
 'What are the two types of charges levied on the policy, as mentioned in the context?',
 'What are the intervals for receiving Loyalty Additions in the policy?',
 'When are Wealth Boosters added to the policy, and how often do they recur?',
 'What options are available for Systematic Withdrawal and Milestone Withdrawal in the policy?',
 'How many funds are available to choose from in the policy, and what are the Portfolio Management Options?',
 'What are the flexible Premium paying options mentioned in the document?',
 'What are the entry age and maturity age ranges for the "Life Option" under the plan?',
 'How does the "Care Option" ensure that savings contributions continue in the event of the life assured\'s absence?',
 'What is the policy term range for the "Regular Pay" option under the "Century Option"?',
 'In the "Life Option," what is payable upon the death of the life assured if there have been partial withdrawals?',
 'What happens to the fund value at the maturity of the policy under the "Care Option"?',
 'What is the Century Option and what does it cover?',
 'What happens if the Life Assured survives until the policy anniversary when they turn 100 years old?',
 'What types of withdrawals can the Life Assured make prior to their death?',
 'When can a Cover Option be chosen, and can it be changed later during the Policy Term?',
 'What is the payable amount on the death of the Life Assured under the Century Option?',
]

In [3]:
%pip install -qU langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [23]:
from langchain_huggingface import HuggingFaceEmbeddings
#from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/Applications/anaconda3/envs/misenv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Applications/anaconda3/envs/misenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
%pip install -qU faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [19]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(fourg_chunks, embeddings)

NameError: name 'fourg_chunks' is not defined

In [14]:
retriever=db.as_retriever()

In [5]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1:8b",
)

In [15]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [16]:

# Generate answers for each context and its corresponding questions
import time
results = []
for question in questions:
    result = rag_chain.invoke({"input": question})
    results.append(result["answer"])
    time.sleep(2)

RemoteProtocolError: Server disconnected without sending a response.

In [26]:
result = rag_chain1.invoke({"input": 'When do the additions start in the policy, and what are the conditions for their availability? '})
print(result['answer'])

According to the context, there is no mention of "additions" but I believe you might be referring to Systematic Withdrawal (SWO) and other features like Premium Redirection, Switching, etc.

For SWO, it can be exercised at any time before the payout starts. Once started, changes or switching cannot be made. 

As for the other features mentioned (Premium Redirection, Switching, Partial withdrawal, Systematic Transfer Option, Return Protector Option, Auto Fund Rebalancing, Safety Switch Option, Settlement Option, Surrender), they can be chosen and changed up to anytime before the payout starts.


In [18]:
for answer in results:
    print(answer)
    print()

According to the context, the following charges will be deducted from the policy: 

1. Mortality Charges 
2. Premium Funding Benefit (PFB) Charges 
3. Partial Withdrawal Charges 
4. Switches Charges 
5. Miscellaneous Charges 
6. Surrender/Discontinuance Charges (applicable for policies with single premium up to Rs. 3,00,000 or above Rs. 3,00,000)

Yes, according to the context, there is an option for a refund of Mortality Charges available in all three options of the Invest 4G plan.

The Loyalty Additions start from the end of 5th Policy Year from the date of commencement and every 5th Policy Year thereafter i.e., 10th, 15th, 20th Policy Year etc. till the end of Premium Paying Term.

Wealth Boosters are available at the end of 10th Policy Year from the date of commencement and every 5th Policy Year i.e., 15th, 20th Policy year etc. thereafter till the end of Policy Term.

The policy prohibits partial withdrawals, SWO, MWO, RPO, SSO, and AFR during the Settlement Period. The Lock-in pe

In [10]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchain-chroma beautifulsoup4

In [31]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

### Construct retriever ###
'''loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)'''
vectorstore = Chroma.from_documents(documents=fourg_chunks, embedding=embeddings)
retriever = vectorstore.as_retriever()


### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
system_prompt3 = (
    "You are an assistant for question-answering tasks based on a specific domain. "
    "Your knowledge base is just the retrieved context. Do not give the answer to the question "
    "which is out of the retrieved context pieces. For answering"
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you don't know. "
    "Do not generate the answer when the question asked is not relevant to the context retrieved, in"
    "this case simply say that please ask the relevant questions based on the product. "
    "Strictly stick to the context retrived to answer the question. Do NOT give the answer which is out of the context."
    "\n\n"
    "{context}"
)
system_prompt5 = """
Answer the user's questions based on the below context. When making up something, strictly stick to the facts and numbers based on the below context.
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know, please ask a relevant question":

<context>
{context}
</context>
"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt5),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [32]:
### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [26]:
answer1=conversational_rag_chain.invoke(
    {"input": "What are the three options available under the Invest 4G plan?"},
    config={
        "configurable": {"session_id": "abcd123"}
    },  # constructs a key "abc123" in `store`.
)
print(answer1['answer'])

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


The three options available under the Invest 4G plan are:

1. Life Option
2. Care Option
3. Century Option

These options ensure that your life is fully secured, as per the context retrieved.


In [33]:
#prompt5
answer11=conversational_rag_chain.invoke(
    {"input": "What are the three options available under the Invest 4G plan?"},
    config={
        "configurable": {"session_id": "abcd123"}
    },  # constructs a key "abc123" in `store`.
)
print(answer11['answer'])

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


The three options available under the Invest 4G plan are:

1. Life Option
2. Care Option
3. Century Option


In [27]:
answer2=conversational_rag_chain.invoke(
    {"input": "explain me about these three cover options under these plan"},
    config={"configurable": {"session_id": "abcd123"}},
)
print(answer2['answer'])

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


Based on the context retrieved, here's an explanation of the three cover options available under the Invest 4G plan:

**1. Life Option**: This option provides life coverage for a specified term (10-30 years or till age 100) and pays out the accumulated fund value at maturity, along with higher of Sum Assured less partial withdrawals or Fund Value, subject to 105% of all Premiums paid, in case of death.

**2. Care Option**: This option provides life coverage and ensures that your savings contributions are made even in your absence through a 'Premium Funding Benefit'. A lump sum (higher of Sum Assured or 105% of all Premiums paid) is paid on death, and remaining future premiums are funded by the Company. The Fund Value is payable at maturity.

**3. Century Option**: This option provides life coverage until you turn 100 years old. In case of death, higher of Sum Assured less partial withdrawals or Fund Value, subject to 105% of all Premiums paid, is payable. If you survive till the policy

In [34]:
#prompt5
answer21=conversational_rag_chain.invoke(
    {"input": "explain me about these three cover options under these plan"},
    config={"configurable": {"session_id": "abcd123"}},
)
print(answer21['answer'])

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


Based on the provided text, here's an explanation of each Cover Option:

**Life Option (Cover Option 1)**:
This option provides a Maturity Benefit and a Death Benefit.
- **Maturity Benefit**: The policyholder receives a lump sum equal to the higher of the Sum Assured or 105% of all premiums paid up to the date of maturity, plus accrued investment gains (Fund Value).
- **Death Benefit** (Illustrative Example 3): If the policyholder passes away during the policy term, a lump sum (the higher of the Sum Assured or 105% of all premiums paid up to the date of death) is paid out immediately. The remaining future premiums are funded by the Company, ensuring that the policy's savings continue uninterrupted.

**Care Option (Cover Option 2)**:
This option provides a Maturity Benefit and a Death Benefit.
- **Maturity Benefit**: Similar to Life Option, the policyholder receives a lump sum equal to the higher of the Sum Assured or 105% of all premiums paid up to the date of maturity, plus accrued in

In [10]:
answer=conversational_rag_chain.invoke(
    {"input": "explain me about these three cover options"},
    config={"configurable": {"session_id": "abcd123"}},
)["answer"]
print(answer)

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


"Here's a brief explanation of each of the three Cover Options:\n\n1. **Life Option**: This option provides life coverage for the policy term and pays out the accumulated fund value at maturity. In case of death, the higher of Sum Assured (less partial withdrawals) or Fund Value is payable.\n\n2. **Care Option**: This option also provides life coverage but ensures that future premiums are funded by the company in case of the policyholder's demise. The lump sum (higher of Sum Assured or Fund Value) is paid out, and premiums continue to be paid until maturity.\n\n3. **Century Option**: In this option, if the policyholder survives until maturity (age 100), the fund value at that time is paid out. This option does not provide a payout in case of death before age 100."

In [28]:
from langchain_core.messages import AIMessage

for message in store["abcd123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What are the three options available under the Invest 4G plan?

AI: The three options available under the Invest 4G plan are:

1. Life Option
2. Care Option
3. Century Option

These options ensure that your life is fully secured, as per the context retrieved.

User: explain me about these three cover options under these plan

AI: Based on the context retrieved, here's an explanation of the three cover options available under the Invest 4G plan:

**1. Life Option**: This option provides life coverage for a specified term (10-30 years or till age 100) and pays out the accumulated fund value at maturity, along with higher of Sum Assured less partial withdrawals or Fund Value, subject to 105% of all Premiums paid, in case of death.

**2. Care Option**: This option provides life coverage and ensures that your savings contributions are made even in your absence through a 'Premium Funding Benefit'. A lump sum (higher of Sum Assured or 105% of all Premiums paid) is paid on death, and rem

In [35]:
#prompt5
for message in store["abcd123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What are the three options available under the Invest 4G plan?

AI: The three options available under the Invest 4G plan are:

1. Life Option
2. Care Option
3. Century Option

User: explain me about these three cover options under these plan

AI: Based on the provided text, here's an explanation of each Cover Option:

**Life Option (Cover Option 1)**:
This option provides a Maturity Benefit and a Death Benefit.
- **Maturity Benefit**: The policyholder receives a lump sum equal to the higher of the Sum Assured or 105% of all premiums paid up to the date of maturity, plus accrued investment gains (Fund Value).
- **Death Benefit** (Illustrative Example 3): If the policyholder passes away during the policy term, a lump sum (the higher of the Sum Assured or 105% of all premiums paid up to the date of death) is paid out immediately. The remaining future premiums are funded by the Company, ensuring that the policy's savings continue uninterrupted.

**Care Option (Cover Option 2)**:
This

In [29]:
answer3=conversational_rag_chain.invoke(
    {"input": "give me example for particular age and polciy term through which i can understand the overall benfit of invest4g plan"},
    config={"configurable": {"session_id": "abcd123"}},
)
print(answer3["answer"])

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


Based on the context retrieved, I'll provide an example for a 35-year-old individual to illustrate the benefits of the Invest 4G plan with different policy terms.

Let's consider three scenarios:

**Scenario 1: Life Option with 20-year term**

* Age: 35
* Policy Term: 20 years
* Sum Assured: ₹10 Lakhs
* Premiums paid: ₹5,000 per month (for 240 months)
* Fund Value at Maturity (assuming a 6% annual return): ₹25.11 Lakhs

In this scenario:

* If the individual passes away within the 20-year term, the accumulated fund value (₹25.11 Lakhs) is paid to their nominee, along with higher of Sum Assured less partial withdrawals or Fund Value, subject to 105% of all Premiums paid.
* If the individual survives the 20-year term, the policy matures, and they receive the accumulated fund value (₹25.11 Lakhs).

**Scenario 2: Care Option with 20-year term**

* Age: 35
* Policy Term: 20 years
* Sum Assured: ₹10 Lakhs
* Premiums paid: ₹5,000 per month (for 240 months)
* Fund Value at Maturity (assuming a

In [36]:
#prompt5
answer31=conversational_rag_chain.invoke(
    {"input": "give me example for particular age and polciy term through which i can understand the overall benfit of invest4g plan"},
    config={"configurable": {"session_id": "abcd123"}},
)
print(answer31["answer"])

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


Let's consider a specific example to illustrate how the Invest 4G plan works.

**Example**

Suppose we have a policyholder, Rohan, who is 30 years old. He chooses the Life Option (Cover Option 1) with a Sum Assured of ₹10 lakhs and pays premiums for 20 years until he turns 50. Let's assume he invests ₹50,000 per year in his Invest 4G plan.

**Assumptions**

* The investment fund grows at an annual compound rate of 7%.
* The mortality charges are deducted from the policy value.
* The premium funding benefit is applied to ensure that the targeted savings contributions are made even in Rohan's absence (i.e., if he passes away, his future premiums will be funded by the Company).

**Calculations**

Let's calculate the maturity and death benefits for Rohan under this scenario:

**Maturity Benefit (Age 50)**

* Total premiums paid: ₹10 lakhs × 20 = ₹2 crores
* Fund value at maturity: ₹2 crores × (1 + 7%)^20 ≈ ₹15.47 crores
* Maturity benefit: Higher of Sum Assured (₹10 lakhs) or 105% of all p

In [38]:
answer31['context']

[Document(metadata={'page': 5, 'source': 'invest4g_Product_Brochure.pdf'}, page_content='Annual  Premium  \n(`)Sum AssuredTotal  Premiums Paid \n(`)Total  Maturity  Benefit (`)\n(Fund Valu e) at the end of 15 years\n4%##8%##\n11,15,305 15,46,869\n22,44,078 31,11,471\n33,66,117 4 6,67,206\nDeat hBenefit : In case of Amit ’s unfortunate deathat the en dof 5thpolicy yea r:\n\uf0d8\n\uf0d8\n\uf0d8\nAnnual  PremiumTotal future \nPremium  funded by \nthe Company  (`)Total  Maturity  Beneﬁt (`)\n(Fund Value) at the end of 15 \nyears\n4%##8%##\n11,23,963 15,61,458\n22,61,694 31,40,996\n33,92,541 47,11,494\nIllustrative Example 3:  Mrs. Singh, age 40 years, is a successful business woman and an intelligent investor having \ninvestments in diﬀerent instruments. She chooses Cover Option 1 (Life Option) under this plan, to pay annual premium for a period of 10 years and a Policy Term of 10 years as shown below. \nMaturity Beneﬁt: The table below shows fund values for multiple scenarios assuming an

In [30]:
answer4=conversational_rag_chain.invoke(
    {"input": "in this example what if the policy term was 40 years."},
    config={"configurable": {"session_id": "abcd123"}},
)
print(answer4["answer"])

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


Let me recalculate the examples for a 35-year-old individual with a 40-year policy term:

**Scenario 1: Life Option with 40-year term**

* Age: 35
* Policy Term: 40 years
* Sum Assured: ₹10 Lakhs
* Premiums paid: ₹5,000 per month (for 480 months)
* Fund Value at Maturity (assuming a 6% annual return): ₹51.45 Lakhs

In this scenario:

* If the individual passes away within the 40-year term, the accumulated fund value (₹51.45 Lakhs) is paid to their nominee, along with higher of Sum Assured less partial withdrawals or Fund Value, subject to 105% of all Premiums paid.
* If the individual survives the 40-year term, the policy matures, and they receive the accumulated fund value (₹51.45 Lakhs).

**Scenario 2: Care Option with 40-year term**

* Age: 35
* Policy Term: 40 years
* Sum Assured: ₹10 Lakhs
* Premiums paid: ₹5,000 per month (for 480 months)
* Fund Value at Maturity (assuming a 6% annual return): ₹51.45 Lakhs

In this scenario:

* If the individual passes away within the 40-year ter

In [37]:
#prompt_5
answer41=conversational_rag_chain.invoke(
    {"input": "in this example what if the policy term was 40 years."},
    config={"configurable": {"session_id": "abcd123"}},
)
print(answer41["answer"])

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


Let's recalculate the maturity and death benefits for Rohan under the same scenario, but with a 40-year policy term.

**Example**

Suppose we have a policyholder, Rohan, who is 30 years old. He chooses the Life Option (Cover Option 1) with a Sum Assured of ₹10 lakhs and pays premiums for 40 years until he turns 70. Let's assume he invests ₹50,000 per year in his Invest 4G plan.

**Assumptions**

* The investment fund grows at an annual compound rate of 7%.
* The mortality charges are deducted from the policy value.
* The premium funding benefit is applied to ensure that the targeted savings contributions are made even in Rohan's absence (i.e., if he passes away, his future premiums will be funded by the Company).

**Calculations**

Let's calculate the maturity and death benefits for Rohan under this scenario:

**Maturity Benefit (Age 70)**

* Total premiums paid: ₹10 lakhs × 40 = ₹4 crores
* Fund value at maturity: ₹4 crores × (1 + 7%)^40 ≈ ₹52.34 crores
* Maturity benefit: Higher of S

In [27]:
#before reconstructing the prompt
answer4=conversational_rag_chain.invoke(
    {"input": "in this example what if the policy term was 40 years."},
    config={"configurable": {"session_id": "abcd123"}},
)
print(answer4["answer"])

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


Let's recalculate the benefits for a Policy Term of 40 years:

**Scenario 1: Death Benefit**

* Sum Assured: ₹50 lacs (or 105% of all premiums paid, whichever is higher)
* Premiums Paid: ₹20 lacs (₹50,000 p.a. × 40 years)

If the policyholder dies after 25 years (age 60), and all premiums are paid on time:

1. Fund Value as on the date of death: ₹30 lacs (₹50,000 p.a. × 8% p.a. for 25 years)
2. Death Benefit: Higher of Sum Assured (₹50 lacs) or Fund Value (₹30 lacs): ₹50 lacs
3. Remaining future premiums as on the date of death: ₹5 lacs (₹50,000 p.a. × 40 - 25 = 15 years)
4. Premium Funding Benefit: The Company will fund these remaining premiums.

**Scenario 2: Maturity Benefit**

* Fund Value as on maturity (age 75): ₹45 lacs
* Maturity Benefit: ₹45 lacs

Assuming the policyholder survives till the end of the Policy Term (40 years):

1. Loyalty Additions:
	+ After 5th year: ₹50,000 × 0.10% = ₹500 (on Fund Value)
	+ After 10th year: ₹50,000 × 0.25% = ₹12,500 (on Fund Value)
	+ After 15

In [29]:
print(qa_prompt)

input_variables=['chat_history', 'context', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]


In [13]:
answer=conversational_rag_chain.invoke(
    {"input": "give me example for particular age and polciy term through which i can understand the overall benfit of invest4g plan"},
    config={"configurable": {"session_id": "abcd123"}},
)["answer"]
print(answer)

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


Let's take an example to illustrate the benefits of investing in this plan.

**Example:**

Mr. Kumar, aged 30, chooses Cover Option 1 (Life Option) with a monthly premium of Rs. 50,000 and a Premium Paying Term & Policy Term of 20 years.

**Assumptions:**

* Annual gross investment return: 8%
* Fund Value at the end of 15 years: Rs. 6,63,96,009 (as per the table in the context)

**Benefits:**

1. **Life Cover**: In case of Mr. Kumar's unfortunate death during the policy term, his family will receive a lump sum payment of Rs. 50,00,000 (Sum Assured) or higher of Rs. 6,63,96,009 (Fund Value), whichever is higher.
2. **Accumulated Savings**: At the end of 20 years, Mr. Kumar's Fund Value will be approximately Rs. 31,49,22,901 (assuming an annual gross investment return of 8%).
3. **Maturity Benefit**: On maturity, Mr. Kumar will receive a lump sum payment of Rs. 31,49,22,901.

**Comparison:**

If Mr. Kumar had invested the same amount of money in a fixed deposit or other low-risk investme

In [14]:
answer=conversational_rag_chain.invoke(
    {"input": "give me example for particular age and polciy term through which i can understand the overall benfit of invest4g plan"},
    config={"configurable": {"session_id": "acd123"}},
)["answer"]
print(answer)

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


Let's consider an example:

**Assumptions:**

* Age at entry: 35 years
* Policy Term: 15 years (Regular Pay)
* Monthly Premium: Rs. 3,000
* Fund Value investment in Equity II Fund with a gross annual return of 4%
* Cover Option chosen: Life Option

**Example Calculation:**

| Year | Total Premiums Paid | Fund Value | Loyalty Additions | Wealth Boosters |
| --- | --- | --- | --- | --- |
| 5th year | Rs. 72,000 (3,000 x 24) | Rs. 1,44,000 | 2% of avg fund value = Rs. 28,800 | - |
| 10th year | Rs. 144,000 | Rs. 3,19,200 | 4% of avg fund value = Rs. 50,560 | 5% of avg fund value = Rs. 15,950 |
| 15th year | Rs. 216,000 | Rs. 6,47,800 | 6% of avg fund value = Rs. 75,320 | 10% of avg fund value = Rs. 31,980 |

**Maturity Benefits:**

* Fund Value at maturity (Rs. 6,47,800) + Loyalty Additions and Wealth Boosters (Rs. 2,44,180) = Total Maturity Benefit of approximately Rs. 8,92,000

**Death Benefit (if policyholder passes away before maturity):**

* Higher of Sum Assured (Rs. 9 lakhs) or 105

In [15]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="mistral:latest",
)

In [16]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

### Construct retriever ###
'''loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)'''
vectorstore = Chroma.from_documents(documents=fourg_chunks, embedding=local_embeddings)
retriever = vectorstore.as_retriever()


### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [17]:
answer=conversational_rag_chain.invoke(
    {"input": "give me example for particular age and polciy term through which i can understand the overall benfit of invest4g plan"},
    config={"configurable": {"session_id": "acd123"}},
)["answer"]
print(answer)

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


 The Invest4G Plan is a hypothetical investment plan designed to illustrate its potential benefits. Let's consider a 35-year-old individual named John, who is looking to save for retirement and wants to understand the advantages of the Invest4G Plan.

Policy terms:
1. Minimum Initial Investment: $5,000
2. Additional investments can be made annually at any amount above $1,000.
3. The plan offers a diversified portfolio of mutual funds and ETFs.
4. A guaranteed annual return of 6% after the 5th year of investment.
5. Tax-deferred growth: No capital gains tax or dividend tax is levied on the earnings within the account until withdrawal.
6. Early withdrawal penalty: If John withdraws funds before age 59 1/2, he will be subject to a 10% penalty on the earnings portion of the withdrawal.
7. The plan allows for Roth IRA conversions annually without any additional fees.
8. The Invest4G Plan offers low management fees of 0.3% per annum.

Now let's examine John's potential benefits from this pla

In [18]:
answer=conversational_rag_chain.invoke(
    {"input": "What are the three options available under the Invest 4G plan?"},
    config={
        "configurable": {"session_id": "abcd123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]
print(answer)

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


1. Invest 4G Growth: This option is suitable for investors who want to grow their wealth over a longer investment horizon (5-7 years). It invests in a diversified portfolio of equities and debt, aiming to generate high returns with moderate risk.

2. Invest 4G Income: Designed for investors seeking regular income along with some capital appreciation. This option primarily invests in fixed income instruments like government bonds, corporate bonds, and money market instruments, with a smaller allocation to equities.

3. Invest 4G Balanced: This option offers a mix of growth and income, making it suitable for those who prefer a more balanced approach. It invests in both equities and debt, with an emphasis on capital preservation and regular income generation.

These options under the Invest 4G plan cater to different risk appetites and investment objectives, allowing investors to choose the one that best fits their financial goals.


In [20]:
answer=conversational_rag_chain.invoke(
    {"input": "explain me about these three cover options"},
    config={"configurable": {"session_id": "abcd123"}},
)["answer"]
print(answer)

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


1. Life Option: Offers life coverage for the policy term with the accumulated fund value at maturity. Higher of Sum Assured or 105% of all premiums paid is payable upon death, and the Fund Value is payable at maturity if the Life Assured survives.
2. Care Option: Provides life coverage with a Premium Funding Benefit that ensures premium payments continue even after the policyholder's demise. Higher of Sum Assured or 105% of all premiums paid is payable upon death, and the Fund Value is payable at maturity if the Life Assured survives.
3. Century Option: Offers life coverage for a longer term (up to age 100) with the accumulated fund value at maturity or upon death, whichever is earlier. The policy term can be extended until age 100 in annual increments, and higher of Sum Assured or 105% of all premiums paid is payable upon death, with the Fund Value being payable at maturity if the Life Assured survives.


In [21]:
answer=conversational_rag_chain.invoke(
    {"input": "give me example for particular age and polciy term through which i can understand the overall benfit of invest4g plan"},
    config={"configurable": {"session_id": "abcd123"}},
)["answer"]
print(answer)

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


 To illustrate the potential benefits of the Canara HSBC Life Insurance Invest 4G plan, let's consider a hypothetical scenario for a 30-year-old individual (John) who decides to invest in this policy with a term of 35 years.

1. Premium Payments: John chooses to pay annual premiums for the entire 35 years. The total amount he pays will depend on the selected premium amount. For simplicity, let's assume that John pays an annual premium of Rs. 1,00,000.

2. Loyalty Additions: Over the policy term, John receives loyalty additions at the end of every 5th year (i.e., on the 5th, 10th, 15th, etc., policy anniversaries). Since this is his first investment, he will start receiving these additions from the 5th year onwards. Assuming a Loyalty Addition Percentage of 3% for the selected fund and taking an average fund value of Rs. 10,00,000 over the last 60 monthly policy anniversaries (considering the first 4 years as settling in period), John would receive a Loyalty Addition of Rs. 30,000 at th

In [19]:
store

{'acd123': InMemoryChatMessageHistory(messages=[HumanMessage(content='give me example for particular age and polciy term through which i can understand the overall benfit of invest4g plan'), AIMessage(content=" The Invest4G Plan is a hypothetical investment plan designed to illustrate its potential benefits. Let's consider a 35-year-old individual named John, who is looking to save for retirement and wants to understand the advantages of the Invest4G Plan.\n\nPolicy terms:\n1. Minimum Initial Investment: $5,000\n2. Additional investments can be made annually at any amount above $1,000.\n3. The plan offers a diversified portfolio of mutual funds and ETFs.\n4. A guaranteed annual return of 6% after the 5th year of investment.\n5. Tax-deferred growth: No capital gains tax or dividend tax is levied on the earnings within the account until withdrawal.\n6. Early withdrawal penalty: If John withdraws funds before age 59 1/2, he will be subject to a 10% penalty on the earnings portion of the w

In [4]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.1:8b")

llm.invoke("The first man on the moon was ...")

'...Neil Armstrong. On July 20, 1969, Neil Armstrong became the first person to set foot on the Moon as part of the Apollo 11 mission. As he stepped off the lunar module Eagle and onto the Moon\'s surface, he famously declared: "That\'s one small step for man, one giant leap for mankind."'

In [6]:
for chunk in llm.stream("The first man on the moon was ..."):
    print(chunk, end="|", flush=True)

...|Neil| Armstrong|.| He| stepped| out| of| the| lunar| module| Eagle| and| onto| the| surface| of| the| Moon| on| July| |20|,| |196|9|,| famously| declaring| "|That|'s| one| small| step| for| man|,| one| giant| leap| for| mankind|"| as| he| took| his| historic| first| steps| on| the| Moon|.||

In [11]:
from langchain_ollama import ChatOllama

chat_model = ChatOllama(model="llama3.1:8b")

answer=chat_model.invoke("Who was the first man on the moon?")
print(answer.content)

The first man to walk on the Moon was Neil Armstrong, an American astronaut and engineer. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission.

As he took his historic first steps on the Moon, Armstrong famously declared: "That's one small step for man, one giant leap for mankind."

Armstrong was followed by fellow astronaut Edwin "Buzz" Aldrin, who also walked on the Moon during the same mission. Michael Collins remained in orbit around the Moon in the command module Columbia.

So, to answer your question, Neil Armstrong was indeed the first man on the Moon!


In [13]:
for response in chat_model.stream("Who was the first man on the moon?"):
    print(response.content, end='', flush=True)

The first man to walk on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first person to set foot on another celestial body.

However, it was actually Edwin "Buzz" Aldrin who followed Armstrong out of the lunar module and joined him on the Moon's surface a short time later. Aldrin was also part of the Apollo 11 mission and walked on the Moon alongside Armstrong.

So while Neil Armstrong was the first person to walk on the Moon, he wasn't alone for long!

In [16]:
%pip install --upgrade --quiet rapidocr-onnxruntime

Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("https://arxiv.org/pdf/2103.15348.pdf", extract_images=True)
pages = loader.load()
pages[3].page_content

'4 Z. Shen et al.\nEfficient Data AnnotationC u s t o m i z e d  M o d e l  T r a i n i n gModel Cust omizationDI A Model HubDI A Pipeline SharingCommunity PlatformLa y out Detection ModelsDocument Images \nT h e  C o r e  L a y o u t P a r s e r  L i b r a r yOCR ModuleSt or age & VisualizationLa y out Data Structur e\nFig. 1: The overall architecture of LayoutParser . For an input document image,\nthe core LayoutParser library provides a set of oﬀ-the-shelf tools for layout\ndetection, OCR, visualization, and storage, backed by a carefully designed layout\ndata structure. LayoutParser also supports high level customization via eﬃcient\nlayout annotation and model training functions. These improve model accuracy\non the target samples. The community platform enables the easy sharing of DIA\nmodels and whole digitization pipelines to promote reusability and reproducibility.\nA collection of detailed documentation, tutorials and exemplar projects make\nLayoutParser easy to learn and use